In [ ]:
from prettytable import PrettyTable 
 
def while_loop(cleaned_code): 
    final_code = [] 
    while_idx = None 
    temp_counter = 1 
 
    def new_temp(): 
        nonlocal temp_counter 
        temp = f"t{temp_counter}" 
        temp_counter += 1 
        return temp 
 
    def precedence(operator): 
        if operator in ('/', '*'): 
            return 2 
        elif operator in ('+', '-'): 
            return 1 
        else: 
            return 0 
 
    def apply_operator(operand_stack, operator_stack): 
        operand2 = operand_stack.pop() 
        operand1 = operand_stack.pop() 
        operator = operator_stack.pop() 
        result_temp = new_temp() 
        final_code.append(f"{result_temp} = {operand1} {operator} {operand2}") 
        operand_stack.append(result_temp) 
 
    for i in range(len(cleaned_code)): 
        codeline = cleaned_code[i] 
 
        if 'while' in codeline: 
            while_idx = i 
            start_idx = codeline.index('(') 
            end_idx = codeline.index(')') 
            bool_condn = ''.join(codeline[start_idx:end_idx+1]) 
            final_code.append('if !{} goto({})'.format(bool_condn, None)) 
            while_idx = i 
        elif '}' in codeline: 
            final_code.append('goto({})'.format(while_idx+1)) 
            final_code[while_idx] = final_code[while_idx].replace( 
                'None', str(i+3)) 
            while_idx = None 
        else: 
            if '=' in codeline: 
                parts = codeline.split('=') 
                var_name = parts[0].strip() 
                operation = parts[1].strip() 
 
                if any(op in operation for op in ('+', '-', '*', '/')): 
                    operands = operation.split() 
                    operand_stack = [] 
                    operator_stack = [] 
                    for operand in operands: 
                        if operand in ('+', '-', '*', '/'): 
                            while (operator_stack and precedence(operator_stack[-1]) >= precedence(operand)): 
                                apply_operator(operand_stack, operator_stack) 
                            operator_stack.append(operand) 
                        else: 
                            operand_stack.append(operand) 
                    while operator_stack: 
                        apply_operator(operand_stack, operator_stack) 
                    final_code.append(f"{var_name} = {operand_stack.pop()}") 
                else: 
                    final_code.append(f"{var_name} = {operation}") 
            else: 
                final_code.append(codeline) 
    return final_code 
 
with open('while.txt') as f: 
    code = f.readlines() 
 
print('The Statement is:') 
print(''.join(code)) 
 
cleaned_code = [] 
for i in range(len(code)): 
    if code[i] != '\n': 
        if code[i][-1] == '\n': 
            cleaned_code.append(code[i][:-1].strip()) 
        else: 
            cleaned_code.append(code[i].strip()) 
 
final_code = while_loop(cleaned_code) 
 
final_code.append('END') 
 
print('\nThe Three Code Generated is:') 
x1 = PrettyTable() 
x1.field_names = ['Index', 'Code'] 
for i in range(len(final_code)): 
    x1.add_row([i+1, final_code[i]]) 
print(x1) 